# COMPLEX - MU4IN900 - 2024/2025
## TME des semaines 3 à 5
---
### TARIK EGE EKEN - 21110611
### BARAN AÇIKEL - 2

In [ ]:
import numpy as np

In [ ]:
class Sac:
    def __init__(self, maxpoids):
        self.maxpoids = maxpoids
        self.poidstotale = 0
        self.valeurtotale = 0
        self.objets = []

    def addObject(self, obj):
        self.objects.append(obj)
        self.poidstotale += obj.poids
        self.valeurtotale += obj.val

    def reset(self):
        self.objects = []
        self.poidstotale = 0
        self.valeurtotale = 0

class Object:
    def __init__(self, poids, valeur):
        self.poids = poids
        self.val = valeur

In [ ]:

def rankObjects(o_list):
    res = []
    for i in range(len(o_list)):
        res.append((o_list[i].val/o_list[i].poids, i))
    return sorted(res)
